In [5]:
import pandas as pd

df = pd.read_csv('data/medical/의료기관 수술인원.csv')
df.head(), df.shape, df['지역별'].unique()


(     시점 지역별    수술인원
 0  2006  서울  240090
 1  2006  부산   93308
 2  2006  대구   59194
 3  2006  인천   62404
 4  2006  광주   34139,
 (283, 3),
 array(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '강원', '충북', '충남',
        '전북', '전남', '경북', '경남', '제주', '세종'], dtype=object))

In [6]:
# '시점'을 datetime 형식으로 변환
df['시점'] = pd.to_datetime(df['시점'].astype(str) + '-01-01')

# 인덱스로 설정
yearly_data = df.set_index('시점')

# 지역별로 그룹화 후 리샘플링
monthly_dfs = []

for region, group in yearly_data.groupby('지역별'):
    monthly_df = group.resample('ME').agg({'수술인원': 'mean'}).round(3)
    
    # 보간
    monthly_df = monthly_df.interpolate(method='linear')
    
    # NaN 값 처리
    monthly_df.fillna(0, inplace=True)
    
    # 시점 다시 설정
    monthly_df.reset_index(inplace=True)
    
    # '시점'이 datetime인지 확인 후, 형식 변경
    if pd.api.types.is_datetime64_any_dtype(monthly_df['시점']):
        monthly_df['시점'] = monthly_df['시점'].dt.strftime('%Y.%m')
    
    # 지역별 컬럼 추가
    monthly_df['지역별'] = region
    
    monthly_dfs.append(monthly_df)

# 모든 지역별 데이터를 하나의 DataFrame으로 결합
final_monthly_df = pd.concat(monthly_dfs, ignore_index=True)
final_monthly_df['수술인원'] = final_monthly_df['수술인원'].map(lambda x: f"{x:.0f}")

# 결과 확인
print(final_monthly_df.head(12), final_monthly_df.shape, final_monthly_df['지역별'].unique())

# 데이터 저장
final_monthly_df.to_csv('data/medical/수술인원(월별).csv', index=False)


         시점   수술인원 지역별
0   2006.01  39240  강원
1   2006.02  39376  강원
2   2006.03  39513  강원
3   2006.04  39649  강원
4   2006.05  39786  강원
5   2006.06  39922  강원
6   2006.07  40058  강원
7   2006.08  40195  강원
8   2006.09  40331  강원
9   2006.10  40468  강원
10  2006.11  40604  강원
11  2006.12  40741  강원 (3209, 3) ['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '세종' '울산' '인천' '전남' '전북'
 '제주' '충남' '충북']
